In [86]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, applications
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Function to extract spectrogram from audio file and convert it to an image-like format
def extract_spectrogram(audio_path, target_size=(224, 224)):
    try:
        audio, sr = librosa.load(audio_path, sr=None)
        spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
        # Convert the spectrogram to dB scale
        spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
        # Resize the image to the target size
        spectrogram_image_resized = tf.image.resize(np.expand_dims(spectrogram_db, axis=-1), target_size)
        return spectrogram_image_resized.numpy()
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

# Path to the folder containing audio files
audio_folder_path = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original"

# List of category labels
category_labels = os.listdir(audio_folder_path)

In [87]:
category_labels

['disco',
 'metal',
 'reggae',
 'blues',
 'rock',
 'classical',
 'jazz',
 'hiphop',
 'country',
 'pop']

In [ ]:
# Extract spectrograms for each audio file using ResNet-50
features_list = []
labels_list = []
for label in category_labels:
    label_folder = os.path.join(audio_folder_path, label)
    for audio_file in os.listdir(label_folder):
        audio_path = os.path.join(label_folder, audio_file)
        features = extract_spectrogram(audio_path, target_size=(224, 224))
        if features is not None:
            # Duplicate the single channel to create three channels
            features_rgb = np.concatenate([features, features, features], axis=-1)
            features_list.append(features_rgb)
            labels_list.append(category_labels.index(label))

# Convert lists to numpy arrays
features_array = np.array(features_list)
labels_array = np.array(labels_list)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    features_array, labels_array, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Build and compile the ResNet-50 model
base_model = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Choose the number of layers to unfreeze
num_layers_to_unfreeze = 40

# Unfreeze layers from the top
for layer in base_model.layers[:-num_layers_to_unfreeze]:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(len(category_labels), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Train model
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_val, y_val),
    verbose=0
)


In [ ]:
# Evaluate model on the test set
predictions_prob = model.predict(X_test)
predictions = np.argmax(predictions_prob, axis=1)

# Calculate accuracy and display results
accuracy = accuracy_score(y_test, predictions)
print("Accuracy of the model:", accuracy)

# Plot training history
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Training Set", "Validation Set"])

# Confusion Matrix
fig, ax = plt.subplots(figsize=(12, 8))
conf_matrix = confusion_matrix(y_test, predictions)
ConfusionMatrixDisplay(conf_matrix, display_labels=category_labels).plot(ax=ax)

# Classification Report
report = classification_report(y_test, predictions)
print('\nClassification Report:\n', report)

In [ ]:
model.save('dl_model2.h5')